In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [4]:
data=pd.read_csv('Churn_Modelling.csv')
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

gender_label_encoder = LabelEncoder()
data['Gender'] = gender_label_encoder.fit_transform(data['Gender'])

geography_onehot_encoder = OneHotEncoder(handle_unknown='ignore')
geography_encoded = geography_onehot_encoder.fit_transform(data[['Geography']]).toarray()
geography_onehot_encoder_df = pd.DataFrame(geography_encoded, columns=geography_onehot_encoder.get_feature_names_out(['Geography']))

data = pd.concat([data.drop('Geography', axis=1), geography_onehot_encoder_df], axis=1)

x = data.drop('Exited', axis=1)
y = data['Exited']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Save encoders and scaler for later use
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(gender_label_encoder, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(geography_onehot_encoder, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [5]:
## Define a function to create the model and try different parameters(KerasClassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(x_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

    return model



In [6]:
## Create a Keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,verbose=1)

In [7]:

# Define the grid search parameters
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2],
    'epochs': [50, 100]
}

In [8]:
# Perform grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3,verbose=1)
grid_result = grid.fit(x_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 16 candidates, totalling 48 fits


/Users/parthabhang/Desktop/DL/Churn Bank Project/churnenv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/parthabhang/Desktop/DL/Churn Bank Project/churnenv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/parthabhang/Desktop/DL/Churn Bank Project/churnenv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/parthabhang/Desktop/DL/Churn Bank Project/churnenv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will b

Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2024-12-08 11:27:49.212808: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-08 11:27:49.238811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-08 11:27:49.241117: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-08 11:27:49.242395: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-08 11:27:49.250623: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-08 11:27:49.268963: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-12-08 11:27:49.289821: I tensorflow/core/grappler/optimizers/cust

167/167 [==============================] - 7s 18ms/step - loss: 0.5371 - accuracy: 0.7459
Epoch 2/50
167/167 [==============================] - 7s 18ms/step - loss: 0.5215 - accuracy: 0.7555
Epoch 2/50
167/167 [==============================] - 8s 21ms/step - loss: 0.5736 - accuracy: 0.7152
Epoch 2/50
167/167 [==============================] - 2s 12ms/step - loss: 0.4435 - accuracy: 0.8076
Epoch 3/50
167/167 [==============================] - 2s 12ms/step - loss: 0.4567 - accuracy: 0.8044
Epoch 3/50
167/167 [==============================] - 2s 11ms/step - loss: 0.4617 - accuracy: 0.8039
Epoch 3/50
167/167 [==============================] - 2s 12ms/step - loss: 0.4325 - accuracy: 0.8101
Epoch 4/50
167/167 [==============================] - 2s 13ms/step - loss: 0.4318 - accuracy: 0.8099
Epoch 4/50
167/167 [==============================] - 2s 12ms/step - loss: 0.4363 - accuracy: 0.8093
Epoch 4/50
167/167 [==============================] - 2s 13ms/step - loss: 0.4372 - accuracy: 0.8056
E

...

167/167 [==============================] - 4s 16ms/step - loss: 0.4748 - accuracy: 0.7767
Epoch 2/100
167/167 [==============================] - 2s 14ms/step - loss: 0.4857 - accuracy: 0.8005
Epoch 12/100
167/167 [==============================] - 2s 15ms/step - loss: 0.4683 - accuracy: 0.8048
Epoch 13/100
167/167 [==============================] - 2s 14ms/step - loss: 0.4676 - accuracy: 0.8058
Epoch 13/100
167/167 [==============================] - 2s 15ms/step - loss: 0.4410 - accuracy: 0.8024
Epoch 3/100
167/167 [==============================] - 2s 14ms/step - loss: 0.4430 - accuracy: 0.8048
Epoch 3/100
167/167 [==============================] - 2s 15ms/step - loss: 0.5164 - accuracy: 0.7926
Epoch 13/100
167/167 [==============================] - 2s 14ms/step - loss: 0.4697 - accuracy: 0.8024
Epoch 14/100
167/167 [==============================] - 2s 15ms/step - loss: 0.4558 - accuracy: 0.8026
Epoch 14/100
167/167 [==============================] - 3s 15ms/step - loss: 0.4673 - acc

/Users/parthabhang/Desktop/DL/Churn Bank Project/churnenv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
2024-12-08 11:46:12.394785: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-12-08 11:46:12.395276: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-12-08 11:46:12.396402: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-12-08 11:46:12.396476: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-12-08 11:46:12.396501: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:

Epoch 1/50


2024-12-08 11:46:13.443073: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


250/250 [==============================] - 3s 4ms/step - loss: 0.5271 - accuracy: 0.7508
Epoch 2/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4362 - accuracy: 0.8115
Epoch 3/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4339 - accuracy: 0.8102
Epoch 4/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4339 - accuracy: 0.8111
Epoch 5/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4342 - accuracy: 0.8109
Epoch 6/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4334 - accuracy: 0.8108
Epoch 7/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4337 - accuracy: 0.8111
Epoch 8/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4336 - accuracy: 0.8111
Epoch 9/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4342 - accuracy: 0.8102
Epoch 10/50
250/250 [==============================] - 1s 4ms/step - loss: 0.4340 - accuracy: 0.8110
Epoch 11/5